# Расстояние Левенштейна. Исправление опечаток

Загрузим необходимые библиотеки

In [5]:
import json
import string
from nltk.tokenize import word_tokenize

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Загрузим корпус

In [4]:
with open('/content/drive/MyDrive/Colab Notebooks/комп линг/corpus_cleared.json', 'r') as corpus_file:
  corpus = json.loads(corpus_file.readline())
corpus['0']

{'comments_ids': [1, 2],
 'comments_number': 2,
 'tags': [],
 'text': 'По просьбам трудящихся организовали паблик анкетных знакомств с той же политикой, что и здесь. До 14-го февраля осталось недолго, но, может, вам удастся найти себе компанию на этот день! ',
 'type': 'post',
 'words_number': 30}

Проведем предобработку:

* разделим текст на токены
* переведем их в нижний регистр
* удалим знаки препинания

Создадим словарь, где ключами будут являться слова из корпуса, а значениями - списки идентификаторов документов, где они встречаются.

In [7]:
punct_marks = string.punctuation + "—" + "«" + "»"

In [14]:
words_docs_dict = dict()

for doc_id in corpus:
  text = word_tokenize(corpus[doc_id]['text'])
  text = [token.lower() for token in text if token not in punct_marks]

  for word in text:
    if word in words_docs_dict:
      if doc_id not in words_docs_dict[word]:
        words_docs_dict[word].append(doc_id)
    else:
      words_docs_dict[word] = [doc_id]

In [21]:
words_docs_dict['здесь']

['0', '218', '268', '378', '462']

Напишем функцию, которая будет принимать слово и возвращать:
* список идентификаторов документов, где оно встречается, если слово есть в словаре  
* иначе - ближайшее к нему слово по расстоянию Левенштейна и список документов, где оно встречается

Воспользуемся реализацией поиска расстояния Левенштейна из библиотеки NLTK.

In [22]:
from nltk.metrics import edit_distance

In [60]:
def word_and_docs_where_to_find_it(word: str):
  if word in words_docs_dict:
    return True, words_docs_dict[word]

  min_dist = 1000
  correct_word = ''
  for corpus_word in words_docs_dict:
    this_dist = edit_distance(word, corpus_word)
    if this_dist < min_dist:
      min_dist = this_dist
      correct_word = corpus_word
  
  if min_dist > 2:
    return False, correct_word, 'другое слово'
  return False, correct_word, 'опечатка', words_docs_dict[correct_word]

In [63]:
word_and_docs_where_to_find_it('здесь')

(True, ['0', '218', '268', '378', '462'])

In [64]:
word_and_docs_where_to_find_it('здес')

(False, 'здесь', 'опечатка', ['0', '218', '268', '378', '462'])

In [65]:
word_and_docs_where_to_find_it('наврено')

(False, 'наверно', 'опечатка', ['141'])

In [66]:
word_and_docs_where_to_find_it('искуственый')

(False, 'естественный', 'другое слово')